# Декораторы II

In [34]:
import functools

In [35]:
def deco(func):
    @functools.wraps(func)
    def wrapped():
        pass
    return wrapped

@deco
def foo():
    print('Hey')
    
    
foo.__name__

'foo'

In [36]:
def validate_user_id(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        if 'user_id' not in kwargs or kwargs['user_id'] == 0:
            print('user_id is wrong')
            return

        print('user_id is correct')
        return func(*args, **kwargs)
    
    return wrapped


def log_access(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        print('{} was called'.format(func.__name__))
        return func(*args, **kwargs)
    return wrapped


@log_access
@validate_user_id
def feedback(user_id, message):
    print('Feedback received')


feedback = log_access(validate_user_id(feedback))
    
data = {
    'message': 'Your app is awesome!',
    'user_id': 42
}

feedback(**data)

feedback was called
user_id is correct
Feedback received


In [37]:
def bold(func):
    def wrapped():
        return "<b>" + func() + "</b>"
    return wrapped


def italic(func):
    def wrapped():
        return "<i>" + func() + "</i>"
    return wrapped


@bold
@italic
def hello():
    return "hello world"


# hello = bold(italic(hello))


print(hello())

<b><i>hello world</i></b>


# Итераторы


In [38]:
iterator = iter([1, 2, 3])

In [42]:
print(next(iterator))

StopIteration: 

In [43]:
class EvenIterator:
    def __init__(self, start, end):
        self.current = start
        self.end = end
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current > self.end:
            raise StopIteration

        self.current += 2
        return self.current - 2
    
    
for num in EvenIterator(1, 5):
    print(num)
        

1
3
5


In [45]:
class IndexIterable:
    def __init__(self, obj):
        self.obj = obj
        
    def __getitem__(self, index):
        return self.obj[index]


for letter in IndexIterable([1, 2, 3]):
    print(letter)

    
# Какой еще объект может быть вместо строки?

1
2
3


In [46]:
import collections



print(isinstance(EvenIterator(1, 10), collections.Iterable))
print(isinstance(IndexIterable('123'), collections.Iterable))


True
False


# Асинхронные итераторы

In [47]:
import asyncio


class Ranger:
    def __init__(self, limit):
        self.limit = limit

    def __aiter__(self):
        return self

    async def __anext__(self):
        await asyncio.sleep(1)
        if self.limit == 0:
            raise StopAsyncIteration
            
        self.limit -= 1
        return self.limit + 1
       

async def main():
    async for val in Ranger(10):
        print(val)

    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

10
9
8
7
6
5
4
3
2
1


# Генераторы

### Что такое генераторы и зачем они нужны?

In [48]:
def fibonacci(number):
    a = b = 1
    for _ in range(number):
        yield a
        a, b = b, a + b
        

for num in fibonacci(100):
    print(num)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025
20365011074
32951280099
53316291173
86267571272
139583862445
225851433717
365435296162
591286729879
956722026041
1548008755920
2504730781961
4052739537881
6557470319842
10610209857723
17167680177565
27777890035288
44945570212853
72723460248141
117669030460994
190392490709135
308061521170129
498454011879264
806515533049393
1304969544928657
2111485077978050
3416454622906707
5527939700884757
8944394323791464
14472334024676221
23416728348467685
37889062373143906
61305790721611591
99194853094755497
160500643816367088
259695496911122585
420196140727489673
679891637638612258
1100087778366101931
1779979416004714189
2880067194370816120
4660046610375530309
7540113804746346429


In [ ]:
def list_generator(list_obj):
    for item in list_obj:
        yield item
        print('After yielding {}'.format(item))


generator = list_generator([1, 2, 3])

In [ ]:
# print(next(generator))

In [ ]:
def accumulator():
    total = 0
    while True:
        value = yield total
        if not value:
            break

        total += value
        

g = accumulator()

# next(g)

# print(g.send(1))
# print(g.send(2))
# print(g.send(3))

# next(g)

In [ ]:
def catcher():
    try:
        while True:
            yield 1

    except ValueError:
        while True:
            yield 2
    

g = catcher()

next(g)

g.throw(ValueError)

next(g)

# Асинхронные генераторы

In [49]:
async def arange(start, stop):
    current = start
    while current <= stop - 1:
        yield current
        current += 1


async def main():
    async for num in arange(10, 20):
        print(num)


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

10
11
12
13
14
15
16
17
18
19


# Списочные выражения

In [50]:
import random


def get_random_list(length):
    random_list = []
    for _ in range(length):
        random_list.append(random.random())
    
    return random_list


print(get_random_list(10))

[0.031861008399387436, 0.07227343717183776, 0.7121876177401635, 0.11798502506453878, 0.26810587880014325, 0.7530465069153776, 0.36873679754299704, 0.0902743148824634, 0.12727493793687195, 0.472525474179046]


In [51]:
import random


def get_random_list(length):
    return [random.random()] * length

print(get_random_list(10))

[0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574, 0.8933286424094574]


In [55]:
import random

[num ** 2 for num in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [56]:
[str(num) for num in range(10)]

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [57]:
[str(num) for num in range(10) if num % 2]

['1', '3', '5', '7', '9']

In [ ]:
[str(num) if num % 2 else num for num in range(10)]

In [58]:
{num % 10 for num in range(100)}

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [59]:
{num: num % 3 for num in range(20)}

{0: 0,
 1: 1,
 2: 2,
 3: 0,
 4: 1,
 5: 2,
 6: 0,
 7: 1,
 8: 2,
 9: 0,
 10: 1,
 11: 2,
 12: 0,
 13: 1,
 14: 2,
 15: 0,
 16: 1,
 17: 2,
 18: 0,
 19: 1}

In [60]:
dict((x, x ** 2) for x in range(4))

{0: 0, 1: 1, 2: 4, 3: 9}

In [61]:
(num ** 2 for num in range(10))

<generator object <genexpr> at 0x107eb2ba0>

In [ ]:
[i * j for i in range(5) for j in range(5, 10)]

In [ ]:
[[x for x in range(5)] for _ in range(5)]

In [ ]:
[sorted(x) for x in [[2, 1], [4, 3], [0, 1]]]

In [ ]:
[print(x) for x in (1, 2, 3)]

In [62]:
a = [x ** 2 for x in range(1, 5)]
b = [x ** 3 for x in range(1, 5)]

list(zip(a, b))

[(1, 1), (4, 8), (9, 27), (16, 64)]

In [63]:
list(zip(a * 2, b))

[(1, 1), (4, 8), (9, 27), (16, 64)]

# Асинхронные comprehensions

In [64]:
async def main():
    return [num async for num in arange(10, 20)]

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

# Контекстные менеджеры

In [ ]:
with open('access_log.log', 'a') as f:
    f.write('New Access\n')


In [65]:
class open_file:
    def __init__(self, filename, mode):
        self.f = open(filename, mode)
    
    def __enter__(self):
        return self.f
    
    def __exit__(self, *args):
        self.f.close()
        
        
with open_file('test.log', 'a') as f:
    f.write('lalala\n')
    
with open_file('test.log', 'r') as f:
    print(f.readlines())

['lalala\n']


In [70]:
import time

time.sleep(1)

In [66]:
class suppress_exception:
    def __init__(self, exc_type):
        self.exc_type = exc_type
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type == self.exc_type:
            print('Nothing happend.')
            return True
    
    
with suppress_exception(ZeroDivisionError) as f:
    print(type(f).__name__)
    big_number = 1 / 0


suppress_exception
Nothing happend.


In [67]:
import contextlib


@contextlib.contextmanager
def printer(num):
    print('Enter')
    
    yield num ** 3
    
    print('Exit')

    
with printer(2) as p:
    print(p)

Enter
8
Exit


## Напишите контекстный менеджер который считает и выводит время, проведенное внутри него

In [71]:
import time

In [76]:

class timer():
    def __init__(self):
        self.start = time.time()

    def time_elapsed(self):
        return time.time() - self.start
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args):
        print(time.time() - self.start)
        
        
with timer() as t:
    time.sleep(2)
    
    print('Elapsed: ', t.time_elapsed())
    
    time.sleep(2)

Elapsed:  2.0044119358062744
4.009259939193726


## Как добавить возможность получить текущее пройденное время?

# Асинхронные контекстные менеджеры

In [1]:
import aiohttp
import asyncio


async def fetch(client):
    async with client.get('http://python.org') as resp:
        assert resp.status == 200
        return await resp.text()


async def main(loop):
    async with aiohttp.ClientSession(loop=loop) as client:
        html = await fetch(client)
        print(html[:96])

        
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->


# Дескрипторы

In [2]:
class Descriptor:
    def __get__(self, obj, obj_type):
        print('get')
        
    def __set__(self, obj, value):
        print('set')


class Class:
    attr = Descriptor()
    

instance = Class()

instance.attr
instance.attr = 10

get
set


In [6]:
class Value:
    def __init__(self):
        self.value = None

    def __get__(self, obj, obj_type):
        return self.value
    
    def __set__(self, obj, value):
        self.value = value * 10
    
class Class:
    attr = Value()

instance = Class()
instance.attr = 100

print(instance.attr)
print(Class.attr)

1000
1000


In [ ]:
# Что происходит в момент доступа к атрибуту — instance.attr (Чтение)

def pseudo_read():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__get__') and (
            hasattr(obj_type, '__set__') or 'attr' not in instance.__dict__
        ):
            return obj_type.__get__(obj, instance, instance.__class__)

    return instance.__dict__['attr']   


# 1. Data Descriptors
# 2. instance.__dict__
# 3. Non-Data Descriptors
# 4. class.__dict__
# 5. base classes


# instance.attr => type(instance).__dict__['attr'].__get__(instance, type(instance))
# Class.attr => Class.__dict__['attr'].__get__(None, Class)
    

In [ ]:
# Что происходит во время — instance.attr = 10 (Записи)

def pseudo_write():
    if hasattr(instance.__class__, 'attr'):
        obj = instance.__class__.attr
        obj_type = obj.__class__

        if hasattr(obj_type, '__set__'):
            obj_type.__set__(obj, instance, 10)
            return

    instance.__dict__['attr'] = 10


## Напишите дескриптор, который пишет в переданный ему во время инициализации файл все присваиваемые значения

In [26]:
class Logged():
    def __init__(self, filename):
        self.filename = filename
        self.value = None
        
    def __get__(self, *args):
        return self.value
    
    def __set__(self, obj, value):
        with open(self.filename, 'a') as f:
            f.write(str(value))

        self.value = value
    
class Class():
    attr = Logged('test1.log')
    
obj = Class()

obj.attr = 10
print(open('test1.log').readlines())

['10101010101010101010101010101010']


### Функции — это дескрипторы

In [27]:
class Class:
    def method(self):
        pass
    
    
obj = Class()    
print(obj.method)
print(Class.method)

<bound method Class.method of <__main__.Class object at 0x111661f28>>
<function Class.method at 0x1116602f0>


In [29]:
def summator(x, y):
    return x + y


lolwhat = summator.__get__(10)
lolwhat(21)

31

### @classmethod & @staticmethod & @property — это дескрпиторы

In [35]:
class Property:
    def __init__(self, getter):
        self.getter = getter

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
    
        return self.getter(obj)
    
class Class:
    @Property
    def y(self):
        return 'y'
    
    def get_z(self):
        return 'z'
    
    z = Property(get_z)
    
obj = Class()

print(obj.y)
print(obj.z)

y
z


## Напишите реализацию @staticmethod или @classmethod

In [42]:
class ClassMethod():
    def __init__(self, func):
        self.func = func
        
    def __get__(self, obj, obj_type=None):
        klass = obj_type or obj.__class__
        def new_func(klass, *args):
            return self.func(*args)
        
        return new_func    
    
class A():
    @ClassMethod
    def send(cls, message):
        print(cls)
        print(message)

a = A()

a.send('Hello, Instance')
A.send('Hello, Class')

TypeError: send() missing 2 required positional arguments: 'cls' and 'message'

# \_\_slots\_\_ (и здесь не без дескрипторов)

In [43]:
class Class:
    __slots__ = ['hello']
    
    def __init__(self):
        self.hello = 'test'

        
obj = Class()

# Мета-классы

### Классы — это объекты

In [44]:
class A:
    ...


In [45]:
a = A()

print(type(a))
print(type(A))

print(isinstance(a, A))
print(isinstance(A, type))

print(issubclass(A, type))
print(issubclass(A, object))

<class '__main__.A'>
<class 'type'>
True
True
False
True


![](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/instance-of.png)


In [46]:
def dummy_factory():
    class Class:
        pass
    
    return Class


Dummy = dummy_factory()
obj = Dummy()

In [48]:
class Meta(type):
    def __new__(cls, name, parents, attrs):
        print('Creating {}'.format(name))
        if 'class_id' not in attrs:
            attrs['class_id'] = name.lower()

        return super().__new__(cls, name, parents, attrs)
    

class A(metaclass=Meta):
    pass


A.class_id

Creating A


'a'

In [49]:
class Meta(type):
    def __init__(cls, name, bases, attrs):
        print('Initializing — {}'.format(name))
        if not hasattr(cls, 'registry'):
            cls.registry = {}
        else:
            cls.registry[name.lower()] = cls
            
        super().__init__(name, bases, attrs)
        
        
class Base(metaclass=Meta):
    pass


class A(Base):
    pass


class B(Base):
    pass


Base.registry

Initializing — Base
Initializing — A
Initializing — B


{'a': __main__.A, 'b': __main__.B}

In [50]:
from abc import ABC, abstractmethod


class Abstract(ABC):
    @abstractmethod
    def abstract_method(self):
        pass


Abstract()

TypeError: Can't instantiate abstract class Abstract with abstract methods abstract_method

# Несколько полезных модулей стандартной библиотеки

### itertools

In [ ]:
import itertools

In [ ]:
# list(itertools.chain(range(10), range(10, 20)))

In [ ]:
# list(itertools.combinations(range(5), 3))

In [ ]:
# list(itertools.permutations(range(3), 3))

In [ ]:
# g = itertools.cycle('ABC')
# for _ in range(10):
#     print(next(g))

In [ ]:
for number in itertools.count():
    if number > 10:
        break
    
#    print(number)

In [ ]:
# list(itertools.repeat(1, 10))

In [ ]:
# list(itertools.product('ABC', 'xy'))

In [ ]:
# list(itertools.zip_longest(range(10), range(5), fillvalue=None))

### re

In [ ]:
import re

In [ ]:
# sentence = 'Call me ASAP!!!11 8-990-1122239 srsly.'
# re.search(r' ([\d-]+) ', sentence).groups()

In [ ]:
# re.findall(r"(\w+)", "Python is the capital of Great Britain")

In [ ]:
# re.search(r"(123)", "a123zzb").group(1)

In [ ]:
# re.sub('(\.)', ' :)', 'Hello...')

In [ ]:
# ints = re.compile(r"(\d+)")
# re.findall(ints, "a123zaa223zb")

### collections

In [ ]:
import collections

In [ ]:
colors = ['red', 'blue', 'red', 'green', 'blue', 'blue']
counter = collections.Counter(colors)

# print(counter)

In [ ]:
# print(counter.most_common(2))

In [ ]:
d = collections.defaultdict(lambda: 'not here')   
# print(d['missing_key'])

In [ ]:
# collections.OrderedDict()

In [ ]:
class SuperList(collections.abc.MutableSequence):
    pass


### datetime

In [ ]:
from datetime import datetime, timedelta

In [ ]:
midterm = datetime.now() + timedelta(days=7)
# print(midterm.day)

In [ ]:
# print(datetime.now().strftime('%Y / %m / %d'))

In [ ]:
# print(datetime.strptime('2016-10-25', '%Y-%m-%d'))